In [1]:
import numpy as np
from definitions import path_join, make_directory, EXPERIMENTS_DIR, VENSIM_MODELS_DIR, logging

In [2]:
KNOWN_MODEL = 'known model'
UNKNOWN_MODEL = 'unknown model'

In [3]:
model_name = 'teacup'
experiment_name = '{}_recovery'.format(model_name)

mode = UNKNOWN_MODEL
need_retrain = False
seed = 123

In [4]:
import tensorflow as tf

general_params = {
    'phi_h': tf.keras.activations.linear,
    'phi_o': tf.keras.activations.linear,
}

train_params = {
    'learning_rate': 1e-1,
    'epochs_before_decay': 0.1,
    'epochs_count': 50,
    'learning_rate_decay': 1/3,
    'iterations_count': 300,
    'early_stopping_patience': 25,
}

In [5]:
def create_experiment_enviroment(model_name, experiment_name, mode):
    experiment_settings = dict()
    
    experiment_settings['model_name'] = model_name
    experiment_settings['experiment_name'] = experiment_name
    experiment_settings['mode'] = mode
    
    experiment_dir = path_join(EXPERIMENTS_DIR, experiment_name)
    make_directory(experiment_dir)
    experiment_settings['experiment_dir'] = experiment_dir
    
    tf_model_dir = path_join(experiment_dir, 'tf_model')
    make_directory(tf_model_dir)
    experiment_settings['tf_model_dir'] = tf_model_dir
    
    images_dir = path_join(experiment_dir, 'images')
    make_directory(images_dir)
    experiment_settings['images_dir'] = images_dir
    
    log_path = path_join(experiment_dir, 'log.log')
    logging.basicConfig(filename=log_path, level=logging.INFO)
    experiment_settings['log_path'] = log_path

    vensim_model_file = path_join(VENSIM_MODELS_DIR, '{}.mdl'.format(model_name))
    experiment_settings['vensim_model_file'] = vensim_model_file
    
    prn_model_dir = path_join(tf_model_dir, 'prn_model')
    nn_model_dir = path_join(tf_model_dir, 'base_nn_model')
    nn_2l_model_dir = path_join(tf_model_dir, 'nn_2l_model')
    make_directory(prn_model_dir)
    make_directory(nn_model_dir)
    make_directory(nn_2l_model_dir)
    
    experiment_settings['prn_model_dir'] = prn_model_dir
    experiment_settings['nn_model_dir'] = nn_model_dir
    experiment_settings['nn_2l_model_dir'] = nn_2l_model_dir

    return experiment_settings
    
experiment_settings = create_experiment_enviroment(model_name, experiment_name, mode)

In [6]:
from definitions import path_join, DATA_DIR

dataset_dir = path_join(DATA_DIR, model_name)
dataset_file = path_join(dataset_dir, 'dataset.csv')

In [7]:
import pandas as pd

data = pd.read_csv(dataset_file)
dt = 0.03125
stopwords = ['TIME', 'sim_index']
fields = [column for column in data.columns if column not in stopwords]

In [8]:
from module.fd_model.vensim_fd_converter import create_unknown_model

FD = create_unknown_model(fields)
FD.dT = dt

print('dt: {}'.format(dt))

dt: 0.03125


In [9]:
fields = [level for level in FD.names_units_map.keys()]
fields

['Characteristic Time', 'Room Temperature', 'Teacup Temperature']

In [10]:
from sklearn.model_selection import train_test_split


def np_preproc_for_rnn3d(numpy_array, fields):
    x_groups = [group[1][fields].values[:-1] for group in numpy_array]
    y_groups = [group[1][fields].values[1:] for group in numpy_array]
    
    train_X, valid_X, train_y, valid_y = train_test_split(x_groups, y_groups, test_size=0.2, random_state=seed)
    
    train_X = np.concatenate(train_X, axis=0)
    valid_X = np.concatenate(valid_X, axis=0)
    
    train_y = np.concatenate(train_y, axis=0)
    valid_y = np.concatenate(valid_y, axis=0)

    return (train_X, train_y), (valid_X, valid_y)


def generate_train_data(df, fields):
    dataset = df[fields].values
    grouped = df.groupby(['sim_index'])[fields]
    
    return dataset, np_preproc_for_rnn3d(grouped, fields)

In [11]:
_, (prn_train, prn_valid) = generate_train_data(data, fields)

In [12]:
prn_train_X, prn_train_y = prn_train
prn_valid_X, prn_valid_y = prn_valid

In [13]:
prn_train_X.shape

(12000, 3)

In [14]:
import tensorflow as tf
from tensorflow.python.framework import ops

ops.reset_default_graph()

In [15]:
from module.fd_model.fd_rnn_converter import FDRNNConverter
# from module.nn_model import NNModel as NNModelv1
from module.nn_model_tf_v2 import NNModel as NNModelv2
from module.nn_model_with_regularizer import NNModel as NNModelv3

choosed_model = NNModelv2

In [16]:
FDRNN_converter = FDRNNConverter(general_params['phi_h'], general_params['phi_o'])
rnn_model = FDRNN_converter.fd_to_rnn(FD, choosed_model)

In [17]:
FD.levels
# FD.constants
# FD.rates

['Characteristic Time', 'Room Temperature', 'Teacup Temperature']

In [18]:
if need_retrain:
    rnn_model.train(prn_train_X, prn_train_y, (prn_valid_X, prn_valid_y), train_params, experiment_settings['prn_model_dir'])

In [19]:
rnn_model.load(experiment_settings['prn_model_dir'])

In [19]:
import tensorflow as tf

In [21]:
def parse_weights(gate, w, fields, rate_names):
    eps = 1e-4
    equations = []
    for col_idx in range(gate.shape[1]):
        equation = ''
        for row_idx in range(gate.shape[0]):
            if (np.abs(w[row_idx, col_idx]) - eps < 0):
                continue
            if gate[row_idx, col_idx] == 0:
                if w[row_idx, col_idx] > 0:
                    sign = '*' # if w[row_idx, col_idx] > 0 else '/'
                    weight = np.abs(w[row_idx, col_idx])
                    if weight == 1:
                        equation += '{}{}'.format(sign, fields[row_idx])
                    else:
                        equation += '{}{}^{:.1f}'.format(sign, fields[row_idx], weight)
                else:
                    continue
        for row_idx in range(gate.shape[0]):
            if (np.abs(w[row_idx, col_idx]) - eps < 0):
                continue
            if gate[row_idx, col_idx] == 0:
                if w[row_idx, col_idx] <= 0:
                    sign = '/' # if w[row_idx, col_idx] > 0 else '/'
                    weight = np.abs(w[row_idx, col_idx])
                    if weight == 1:
                        equation += '{}{}'.format(sign, fields[row_idx])
                    else:
                        equation += '{}{}^{:.1f}'.format(sign, fields[row_idx], weight)
                else:
                    continue
        for row_idx in range(gate.shape[0]):
            if (w[row_idx, col_idx] == 0):
                continue
            if gate[row_idx, col_idx] == 1:
                sign = '+' if w[row_idx, col_idx] > 0 else '-'
                weight = np.abs(w[row_idx, col_idx])
                if weight == 1:
                    equation += '{}{}'.format(sign, fields[row_idx])
                else:
                    equation += '{}{:.1f}{}'.format(sign, weight, fields[row_idx])

        equations.append((rate_names[col_idx], equation[1:]))
    return equations

In [22]:
rate_names = [rate.name for rate in FD.rates]

In [23]:
rate_names

['in_Characteristic Time',
 'out_Characteristic Time',
 'in_Room Temperature',
 'out_Room Temperature',
 'in_Teacup Temperature',
 'out_Teacup Temperature']

In [24]:
fields

['Characteristic Time', 'Room Temperature', 'Teacup Temperature']

In [25]:
def my_tf_round(x, decimals = 0):
    multiplier = tf.constant(10**decimals, dtype=x.dtype)
    return tf.math.round(x * multiplier) / multiplier

In [29]:
_gate = np.array(my_tf_round(rnn_model.model.gate, 0), dtype=np.float32)
_w = np.array(my_tf_round(rnn_model.model.W_ah, 1), dtype=np.float32)
# print(fields)
parse_weights(_gate, _w, fields, rate_names)

[('in_Characteristic Time', ''),
 ('out_Characteristic Time', ''),
 ('in_Room Temperature', ''),
 ('out_Room Temperature', ''),
 ('in_Teacup Temperature',
  'Characteristic Time^0.5/Teacup Temperature^0.4+0.5Room Temperature'),
 ('out_Teacup Temperature',
  'Characteristic Time^0.8/Room Temperature^0.3+0.4Teacup Temperature')]

In [20]:
initial_value = np.reshape(prn_valid_X[0], [1, prn_valid_X.shape[1]])
initial_value

array([[10., 70., 40.]])

In [21]:
iterations_count = train_params['iterations_count']
if iterations_count == 0:
    iterations_count = X.shape[0] - 1
iterations_count

300

In [22]:
prn_iterative = rnn_model.get_simulation(initial_value, iterations_count, experiment_settings['prn_model_dir'])

In [23]:
prn_iterative.shape

(301, 3)

In [24]:
from arch.base_nn import BaseNN
from arch.base_nn_2layers import BaseNN2Layers

predictor = BaseNN2Layers(prn_train_X.shape[1], prn_train_X.shape[1])

In [20]:
need_train = True

In [21]:
if need_train:
    predictor.train(prn_train_X, prn_train_y, (prn_valid_X, prn_valid_y), train_params, experiment_settings['nn_2l_model_dir'])
    pass

Train on 12000 samples, validate on 3000 samples
Epoch 1/50
12000/12000 [==============================] - 1s 108us/sample - loss: 2422.3589 - mse: 2422.3584 - val_loss: 730.2563 - val_mse: 730.2565
Epoch 2/50
12000/12000 [==============================] - 1s 74us/sample - loss: 415.6646 - mse: 415.6647 - val_loss: 286.1596 - val_mse: 286.1596
Epoch 3/50
12000/12000 [==============================] - 1s 73us/sample - loss: 300.6649 - mse: 300.6650 - val_loss: 256.4600 - val_mse: 256.4599
Epoch 4/50
12000/12000 [==============================] - 1s 70us/sample - loss: 276.8381 - mse: 276.8381 - val_loss: 237.6209 - val_mse: 237.6209
Epoch 5/50
12000/12000 [==============================] - 1s 70us/sample - loss: 253.6700 - mse: 253.6699 - val_loss: 216.7729 - val_mse: 216.7730
Epoch 6/50
12000/12000 [==============================] - 1s 71us/sample - loss: 224.2754 - mse: 224.2753 - val_loss: 185.7588 - val_mse: 185.7588
Epoch 7/50
12000/12000 [==============================] - 1s 74us/

In [25]:
nn_output = predictor.test(prn_train_X, experiment_settings['nn_2l_model_dir'])

In [26]:
nn_output[:5]

array([[ 10.06158 ,  55.009342, 138.93025 ],
       [ 10.059795,  55.00929 , 137.88356 ],
       [ 10.058028,  55.009235, 136.84673 ],
       [ 10.056278,  55.00919 , 135.8198  ],
       [ 10.054562,  55.009144, 134.81259 ]], dtype=float32)

In [27]:
predictor.calculate_trainable_parameters()

24

In [28]:
def calculate_error(required_columns_data, output):
    output = np.array(output)
    error = sum(abs((output-required_columns_data)/required_columns_data))/required_columns_data.shape[0]

    return error

<h3>Train set error</h3>

In [29]:
initial_value = np.reshape(prn_train_X[0], [1, prn_train_X.shape[1]])
train_prn_output = rnn_model.get_simulation(initial_value, iterations_count, experiment_settings['prn_model_dir'])
train_prn_error = calculate_error(prn_train_y[:iterations_count], train_prn_output[1:])
train_prn_error

array([0.        , 0.87314815, 1.64409017])

In [ ]:
initial_value = np.reshape(prn_train_X[0], [1, prn_train_X.shape[1]])
train_nn_output = predictor.get_simulation(initial_value, iterations_count, experiment_settings['prn_model_dir'])
train_nn_error = calculate_error(prn_train_y[:iterations_count], train_nn_output[1:])
train_nn_error

In [30]:
train_prn_output = rnn_model.test(prn_train_X, experiment_settings['prn_model_dir'])
train_prn_error = calculate_error(prn_train_y, train_prn_output)
train_prn_error

array([0.00000000e+00, 0.00000000e+00, 5.85885579e-05])

In [37]:
fields

['Characteristic Time', 'Room Temperature', 'Teacup Temperature']

In [31]:
train_nn_output = predictor.test(prn_train_X, experiment_settings['nn_model_dir'])
train_nn_error = calculate_error(prn_train_y, train_nn_output)
train_nn_error

array([2.66896703, 1.67173387, 0.59876746])

<h3>Test set error</h3>

In [38]:
prn_output = predictor.test(prn_valid_X, experiment_settings['nn_model_dir'])
prn_error = calculate_error(prn_valid_y, prn_output)
prn_error

array([2.73180354, 1.45280095, 0.60584841])

In [39]:
valid_nn_output = rnn_model.test(prn_valid_X, experiment_settings['nn_model_dir'])
valid_nn_error = calculate_error(prn_valid_y, valid_nn_output)
valid_nn_error

array([0.00000000e+00, 0.00000000e+00, 5.73139759e-05])

In [20]:
from arch.lstm import LSTMModel

sequence_size = 5
lstm_model_dir = experiment_settings['nn_model_dir'] + '_lstm_{}seq'.format(sequence_size)

predictor = LSTMModel((sequence_size, prn_train_X.shape[1]), 32, prn_train_X.shape[1])

In [21]:
from sklearn.model_selection import train_test_split


def window_stack(a, stepsize=1, width=2):
    n = a.shape[1]
    omg = lambda i: 1+n+i-width
    _X = [a[:, i:i+width].astype(np.float16) for i in range(0, n - width)]
    _y = [a[:, i].astype(np.float16) for i in range(width, n)]
    del a
    _X = np.stack(_X, axis=1).astype(np.float16)
    _y = np.stack(_y, axis=1).astype(np.float16)

    return _X.reshape(-1, _X.shape[2], _X.shape[3]), _y.reshape(-1, _y.shape[2])

def get_X_y(data, width=7):
    data.fillna(0, inplace=True)
    
    X = np.array(list(data.groupby('sim_index').apply(pd.DataFrame.as_matrix)))
    X, y = window_stack(X, width=width)
    X = X[:, :, 1:]
    y = y[:, 1:]
    
    return X, y

train_ids, valid_ids = train_test_split(data['sim_index'].unique(), test_size=0.2, random_state=123)
rnn_train = data[data['sim_index'].isin(train_ids)]
rnn_train_X, rnn_train_y = get_X_y(rnn_train, sequence_size)

c:\users\hwer\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
c:\users\hwer\appdata\local\programs\python\python36\lib\site-packages\pandas\core\groupby\groupby.py:5088: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  starts, ends)


In [22]:
rnn_valid = data[data['sim_index'].isin(valid_ids)]
rnn_valid_X, rnn_valid_y = get_X_y(rnn_valid, sequence_size)
rnn_valid_X[0]

array([[3.000e+01, 3.000e+01, 1.500e-02, 1.010e+00, 1.250e+00, 1.000e-02],
       [2.947e+01, 3.089e+01, 1.500e-02, 1.010e+00, 1.250e+00, 1.000e-02],
       [2.897e+01, 3.181e+01, 1.500e-02, 1.010e+00, 1.250e+00, 1.000e-02],
       [2.848e+01, 3.278e+01, 1.500e-02, 1.010e+00, 1.250e+00, 1.000e-02],
       [2.803e+01, 3.375e+01, 1.500e-02, 1.010e+00, 1.250e+00, 1.000e-02]],
      dtype=float16)

In [24]:
need_train = True

In [25]:
if need_train:
    predictor.train(rnn_train_X, rnn_train_y, (rnn_valid_X, rnn_valid_y), train_params, lstm_model_dir)

Train on 473600 samples, validate on 118400 samples
Epoch 1/50
473600/473600 [==============================] - 54s 114us/sample - loss: 17171.9019 - mse: 17171.9570 - val_loss: 10123.0150 - val_mse: 10122.9727
Epoch 2/50
473600/473600 [==============================] - 54s 115us/sample - loss: 7680.9649 - mse: 7680.9717 - val_loss: 4630.6640 - val_mse: 4630.6572
Epoch 3/50
473600/473600 [==============================] - 52s 109us/sample - loss: 3643.9718 - mse: 3643.9575 - val_loss: 2127.2930 - val_mse: 2127.2935
Epoch 4/50
473600/473600 [==============================] - 47s 98us/sample - loss: 1722.7693 - mse: 1722.7734 - val_loss: 980.3012 - val_mse: 980.3013
Epoch 5/50
473600/473600 [==============================] - 50s 106us/sample - loss: 971.3384 - mse: 971.3382 - val_loss: 692.0197 - val_mse: 692.0197
Epoch 6/50
473600/473600 [==============================] - 46s 96us/sample - loss: 619.2739 - mse: 619.2742 - val_loss: 378.2780 - val_mse: 378.2783
Epoch 7/50
473600/473600 [